<a href="https://colab.research.google.com/github/LatiefDataVisionary/feature-engineering-college-task/blob/main/Tugas_Kelompok_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Feature Engineering**

Nama kelompok:
-
-
-

## **1. Mengimpor Library yang diperlukan**

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/Classroom/Coba/Iris.csv')
df = pd.read_csv('/content/drive/MyDrive/Sbaba/heart_new.csv')
df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,Yes
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,Yes
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,Yes
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,Yes
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,No
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,No
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,No
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,No


Mendapatkan semua fitur kecuali label, lalu melakukan feature engineering yaitu feature extraction ke fitur tersebut, lalu drop kolom lama yang belum dilakukan pca.

In [ ]:
# Menerapkan PCA pada fitur numerikal untuk mereduksi dimensicol = [i for i in df.columns if i != 'output']
col = [i for i in df.columns if i != 'output']
pca = PCA(n_components=2)
numerical_features = df[col]
pca_features = pca.fit_transform(numerical_features)

# Menambahkan fitur hasil PCA ke DataFrame
pca_df = pd.DataFrame(pca_features, columns=["PCA1", "PCA2"])
df_pca = pd.concat([df, pca_df], axis=1)

# Mendrop kolom lama
df_pca.drop(col, axis=1, inplace=True)
df_pca

,output,PCA1,PCA2
0,Yes,-12.267345,-2.873838
1,Yes,2.690137,39.871374
2,Yes,-42.950214,23.636820
3,Yes,-10.944756,28.438036
4,Yes,106.979053,15.874468
...,...,...,...
298,No,-4.554121,-27.490169
299,No,16.428008,-12.921716
300,No,-51.963811,-13.323798
301,No,-114.755981,-36.435184


Melakukan feature construction yaitu label encoder ke label agar label dapat digunakan di ML model.

In [ ]:
X = df_pca[[i for i in df_pca.columns if i != 'output']]
y = df_pca['output']
print(f"X:{X}")

# Encode string labels into numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y = y_encoded

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X:           PCA1       PCA2
0    -12.267345  -2.873838
1      2.690137  39.871374
2    -42.950214  23.636820
3    -10.944756  28.438036
4    106.979053  15.874468
..          ...        ...
298   -4.554121 -27.490169
299   16.428008 -12.921716
300  -51.963811 -13.323798
301 -114.755981 -36.435184
302  -10.396142  23.302401

[303 rows x 2 columns]


Mencoba ML KNN dan XGBoost terhadap dataset yang sudah dilakukan feature engineering.

In [ ]:
# Convert data ke DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# parameter untuk XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': len(np.unique(y)),
    'max_depth': 4,
    'eta': 0.3,
    'eval_metric': 'mlogloss',
    'seed': 42
}

# Ngetrain XGBoost model
num_round = 50
bst = xgb.train(params, dtrain, num_round)

# Membuat prediksi
y_pred = bst.predict(dtest)
accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi XGBoost Menggunakan PCA adalah : {accuracy*100:.2f}%")

Akurasi XGBoost Menggunakan PCA adalah : 75.41%


In [ ]:
# import KNeighbors ClaSSifier from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# instantiate the model
knn = KNeighborsClassifier(n_neighbors=3)


# fit the model to the training set
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(f"Akurasi KNN Menggunakan PCA adalah :{np.sum(y_pred == y_test)/len(y_test) * 100}%")

Akurasi KNN Menggunakan PCA adalah :63.934426229508205%


Mendapatkan kolom fitur kecuali label, lalu kolom fitur teresbut kita lakukan proses feature engineering yaitu feature construction Standard Scaler ke fitur tersebut.

Melakukan Standar Scaler untuk feature engineeringya

Mendapatkan semua fitur kecuali label, lalu melakukan feature engineering yaitu feature construction Standard Scalar ke fitur tersebut, lalu drop kolom lama yang belum dilakukan Standard Scalar.

In [ ]:
# Melakukan standard scaler
col = [i for i in df.columns if i != 'output']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[col])

# Menambahkan fitur yang sudah diskalakan kembali ke DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=[col])
df_scaled = pd.concat([df, scaled_df], axis=1)

# Mendrop kolom lama
df_scaled.drop(col, axis=1, inplace=True)
df_scaled

,output,"(age,)","(sex,)","(cp,)","(trtbps,)","(chol,)","(fbs,)","(restecg,)","(thalachh,)","(exng,)","(oldpeak,)","(slp,)","(caa,)","(thall,)"
0,Yes,0.952197,0.681005,1.973123,0.763956,-0.256334,2.394438,-1.005832,0.015443,-0.696631,1.087338,-2.274579,-0.714429,-2.148873
1,Yes,-1.915313,0.681005,1.002577,-0.092738,0.072199,-0.417635,0.898962,1.633471,-0.696631,2.122573,-2.274579,-0.714429,-0.512922
2,Yes,-1.474158,-1.468418,0.032031,-0.092738,-0.816773,-0.417635,-1.005832,0.977514,-0.696631,0.310912,0.976352,-0.714429,-0.512922
3,Yes,0.180175,0.681005,0.032031,-0.663867,-0.198357,-0.417635,0.898962,1.239897,-0.696631,-0.206705,0.976352,-0.714429,-0.512922
4,Yes,0.290464,-1.468418,-0.938515,-0.663867,2.082050,-0.417635,0.898962,0.583939,1.435481,-0.379244,0.976352,-0.714429,-0.512922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,No,0.290464,-1.468418,-0.938515,0.478391,-0.101730,-0.417635,0.898962,-1.165281,1.435481,-0.724323,-0.649113,-0.714429,1.123029
299,No,-1.033002,0.681005,1.973123,-1.234996,0.342756,-0.417635,0.898962,-0.771706,-0.696631,0.138373,-0.649113,-0.714429,1.123029
300,No,1.503641,0.681005,-0.938515,0.706843,-1.029353,2.394438,0.898962,-0.378132,-0.696631,2.036303,-0.649113,1.244593,1.123029
301,No,0.290464,0.681005,-0.938515,-0.092738,-2.227533,-0.417635,0.898962,-1.515125,1.435481,0.138373,-0.649113,0.265082,1.123029


Melakukan feature construction yaitu label encoder ke label agar label dapat digunakan di ML model.

In [ ]:
X = df_scaled[[i for i in df_scaled.columns if i != 'output']]
y = df_scaled['output']
print(f"X:{X}")

# Melakukan label encoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y = y_encoded

# Split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X:       (age,)    (sex,)     (cp,)  (trtbps,)   (chol,)    (fbs,)  (restecg,)  \
0    0.952197  0.681005  1.973123   0.763956 -0.256334  2.394438   -1.005832   
1   -1.915313  0.681005  1.002577  -0.092738  0.072199 -0.417635    0.898962   
2   -1.474158 -1.468418  0.032031  -0.092738 -0.816773 -0.417635   -1.005832   
3    0.180175  0.681005  0.032031  -0.663867 -0.198357 -0.417635    0.898962   
4    0.290464 -1.468418 -0.938515  -0.663867  2.082050 -0.417635    0.898962   
..        ...       ...       ...        ...       ...       ...         ...   
298  0.290464 -1.468418 -0.938515   0.478391 -0.101730 -0.417635    0.898962   
299 -1.033002  0.681005  1.973123  -1.234996  0.342756 -0.417635    0.898962   
300  1.503641  0.681005 -0.938515   0.706843 -1.029353  2.394438    0.898962   
301  0.290464  0.681005 -0.938515  -0.092738 -2.227533 -0.417635    0.898962   
302  0.290464 -1.468418  0.032031  -0.092738 -0.198357 -0.417635   -1.005832   

     (thalachh,)   (exng,)  (oldpeak,

Mencoba ML KNN dan XGBoost terhadap dataset yang sudah dilakukan feature engineering.

In [ ]:
# Convert data ke DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# parameter untuk XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': len(np.unique(y)),
    'max_depth': 4,
    'eta': 0.3,
    'eval_metric': 'mlogloss',
    'seed': 42
}

# Ngetrain XGBoost model
num_round = 50
bst = xgb.train(params, dtrain, num_round)

# Membuat prediksi
y_pred = bst.predict(dtest)
accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi XGBoost Menggunakan Standard Scaler adalah : {accuracy*100:.2f}%")

Akurasi XGBoost Menggunakan Standard Scaler adalah : 83.61%


In [ ]:
# import KNeighbors ClaSSifier from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# instantiate the model
knn = KNeighborsClassifier(n_neighbors=3)


# fit the model to the training set
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(f"Akurasi KNN Menggunakan Standard Scaler adalah :{np.sum(y_pred == y_test)/len(y_test) * 100}%")

Akurasi KNN Menggunakan Standard Scaler adalah :86.88524590163934%


Min Max Scaler

Mendapatkan kolom fitur kecuali label, lalu kolom fitur teresbut kita lakukan proses feature engineering yaitu feature construction Min Max Scaler ke fitur tersebut.

Mendapatkan semua fitur kecuali label, lalu melakukan feature engineering yaitu feature construction Min Max Scaler ke fitur tersebut, lalu drop kolom lama yang belum dilakukan Min Max Scaler.

In [ ]:
# Melakukan Min Max Scaler
col = [i for i in df.columns if i != 'output']
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[col])

# Menambahkan fitur yang sudah diskalakan kembali ke DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=[col])
df_scaled = pd.concat([df, scaled_df], axis=1)

# Mendrop kolom lama
df_scaled.drop(col, axis=1, inplace=True)
df_scaled

,output,"(age,)","(sex,)","(cp,)","(trtbps,)","(chol,)","(fbs,)","(restecg,)","(thalachh,)","(exng,)","(oldpeak,)","(slp,)","(caa,)","(thall,)"
0,Yes,0.708333,1.0,1.000000,0.481132,0.244292,1.0,0.0,0.603053,0.0,0.370968,0.0,0.00,0.333333
1,Yes,0.166667,1.0,0.666667,0.339623,0.283105,0.0,0.5,0.885496,0.0,0.564516,0.0,0.00,0.666667
2,Yes,0.250000,0.0,0.333333,0.339623,0.178082,0.0,0.0,0.770992,0.0,0.225806,1.0,0.00,0.666667
3,Yes,0.562500,1.0,0.333333,0.245283,0.251142,0.0,0.5,0.816794,0.0,0.129032,1.0,0.00,0.666667
4,Yes,0.583333,0.0,0.000000,0.245283,0.520548,0.0,0.5,0.702290,1.0,0.096774,1.0,0.00,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,No,0.583333,0.0,0.000000,0.433962,0.262557,0.0,0.5,0.396947,1.0,0.032258,0.5,0.00,1.000000
299,No,0.333333,1.0,1.000000,0.150943,0.315068,0.0,0.5,0.465649,0.0,0.193548,0.5,0.00,1.000000
300,No,0.812500,1.0,0.000000,0.471698,0.152968,1.0,0.5,0.534351,0.0,0.548387,0.5,0.50,1.000000
301,No,0.583333,1.0,0.000000,0.339623,0.011416,0.0,0.5,0.335878,1.0,0.193548,0.5,0.25,1.000000


Melakukan feature construction yaitu label encoder ke label agar label dapat digunakan di ML model.

In [ ]:
X = df_scaled[[i for i in df_scaled.columns if i != 'output']]
y = df_scaled['output']
print(f"X:{X}")

# Melakukan label encoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y = y_encoded

# Split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X:       (age,)  (sex,)     (cp,)  (trtbps,)   (chol,)  (fbs,)  (restecg,)  \
0    0.708333     1.0  1.000000   0.481132  0.244292     1.0         0.0   
1    0.166667     1.0  0.666667   0.339623  0.283105     0.0         0.5   
2    0.250000     0.0  0.333333   0.339623  0.178082     0.0         0.0   
3    0.562500     1.0  0.333333   0.245283  0.251142     0.0         0.5   
4    0.583333     0.0  0.000000   0.245283  0.520548     0.0         0.5   
..        ...     ...       ...        ...       ...     ...         ...   
298  0.583333     0.0  0.000000   0.433962  0.262557     0.0         0.5   
299  0.333333     1.0  1.000000   0.150943  0.315068     0.0         0.5   
300  0.812500     1.0  0.000000   0.471698  0.152968     1.0         0.5   
301  0.583333     1.0  0.000000   0.339623  0.011416     0.0         0.5   
302  0.583333     0.0  0.333333   0.339623  0.251142     0.0         0.0   

     (thalachh,)  (exng,)  (oldpeak,)  (slp,)  (caa,)  (thall,)  
0       0.603053   

Mencoba ML KNN dan XGBoost terhadap dataset yang sudah dilakukan feature engineering.

In [ ]:
# Convert data ke DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# parameter untuk XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': len(np.unique(y)),
    'max_depth': 4,
    'eta': 0.3,
    'eval_metric': 'mlogloss',
    'seed': 42
}

# Ngetrain XGBoost model
num_round = 50
bst = xgb.train(params, dtrain, num_round)

# Membuat prediksi
y_pred = bst.predict(dtest)
accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi XGBoost Menggunakan MinMax Scaler adalah : {accuracy*100:.2f}%")

Akurasi XGBoost Menggunakan MinMax Scaler adalah : 83.61%


In [ ]:
# import KNeighbors ClaSSifier from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# instantiate the model
knn = KNeighborsClassifier(n_neighbors=3)


# fit the model to the training set
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(f"Akurasi KNN Menggunakan Min Max Scaler adalah :{np.sum(y_pred == y_test)/len(y_test) * 100}%")

Akurasi KNN Menggunakan Min Max Scaler adalah :83.60655737704919%


Power Transformer

Mendapatkan kolom fitur kecuali label, lalu kolom fitur teresbut kita lakukan proses feature engineering yaitu Power Transformer ke fitur tersebut.

Mendapatkan semua fitur kecuali label, lalu melakukan feature engineering yaitu feature construction Power Transformer ke fitur tersebut, lalu drop kolom lama yang belum dilakukan Power Transformer.

In [ ]:
# Melakukan Power Transformer
col = [i for i in df.columns if i != 'output']
scaler = PowerTransformer()
scaled_features = scaler.fit_transform(df[col])

# Menambahkan fitur yang sudah diskalakan kembali ke DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=[col])
df_scaled = pd.concat([df, scaled_df], axis=1)
df_scaled

# Mendrop kolom lama
df_scaled.drop(col, axis=1, inplace=True)
df_scaled

,output,"(age,)","(sex,)","(cp,)","(trtbps,)","(chol,)","(fbs,)","(restecg,)","(thalachh,)","(exng,)","(oldpeak,)","(slp,)","(caa,)","(thall,)"
0,Yes,0.956171,0.681005,1.487217,0.831717,-0.159046,2.394438,-1.021730,-0.081881,-0.696631,1.180998,-1.958263,-0.839679,-1.946718
1,Yes,-1.831006,0.681005,1.058192,0.015350,0.187234,-0.417635,0.933925,1.871628,-0.696631,1.592215,-1.958263,-0.839679,-0.583232
2,Yes,-1.442978,-1.468418,0.379389,0.015350,-0.819710,-0.417635,-1.021730,1.015375,-0.696631,0.686518,1.015783,-0.839679,-0.583232
3,Yes,0.147656,0.681005,0.379389,-0.623762,-0.095947,-0.417635,0.933925,1.347213,-0.696631,0.166444,1.015783,-0.839679,-0.583232
4,Yes,0.260722,-1.468418,-1.015652,-0.623762,1.859954,-0.417635,0.933925,0.543973,1.435481,-0.069148,1.015783,-0.839679,-0.583232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,No,0.260722,-1.468418,-1.015652,0.576126,0.007273,-0.417635,0.933925,-1.179283,1.435481,-0.706744,-0.736554,-0.839679,1.176072
299,No,-1.039024,0.681005,1.487217,-1.360101,0.453389,-0.417635,0.933925,-0.843304,-0.696631,0.537070,-0.736554,-0.839679,1.176072
300,No,1.557230,0.681005,-1.015652,0.781806,-1.097969,2.394438,0.933925,-0.477666,-0.696631,1.564525,-0.736554,1.341145,1.176072
301,No,0.260722,0.681005,-1.015652,0.015350,-3.088792,-0.417635,0.933925,-1.453449,1.435481,0.537070,-0.736554,0.871923,1.176072


Melakukan feature construction yaitu label encoder ke label agar label dapat digunakan di ML model.

In [ ]:
X = df_scaled[[i for i in df_scaled.columns if i != 'output']]
y = df_scaled['output']
print(f"X:{X}")

# Melakukan label encoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y = y_encoded

# Split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X:       (age,)    (sex,)     (cp,)  (trtbps,)   (chol,)    (fbs,)  (restecg,)  \
0    0.956171  0.681005  1.487217   0.831717 -0.159046  2.394438   -1.021730   
1   -1.831006  0.681005  1.058192   0.015350  0.187234 -0.417635    0.933925   
2   -1.442978 -1.468418  0.379389   0.015350 -0.819710 -0.417635   -1.021730   
3    0.147656  0.681005  0.379389  -0.623762 -0.095947 -0.417635    0.933925   
4    0.260722 -1.468418 -1.015652  -0.623762  1.859954 -0.417635    0.933925   
..        ...       ...       ...        ...       ...       ...         ...   
298  0.260722 -1.468418 -1.015652   0.576126  0.007273 -0.417635    0.933925   
299 -1.039024  0.681005  1.487217  -1.360101  0.453389 -0.417635    0.933925   
300  1.557230  0.681005 -1.015652   0.781806 -1.097969  2.394438    0.933925   
301  0.260722  0.681005 -1.015652   0.015350 -3.088792 -0.417635    0.933925   
302  0.260722 -1.468418  0.379389   0.015350 -0.095947 -0.417635   -1.021730   

     (thalachh,)   (exng,)  (oldpeak,

Mencoba ML KNN dan XGBoost terhadap dataset yang sudah dilakukan feature engineering.

In [ ]:
# Convert data ke DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# parameter untuk XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': len(np.unique(y)),
    'max_depth': 4,
    'eta': 0.3,
    'eval_metric': 'mlogloss',
    'seed': 42
}

# Ngetrain XGBoost model
num_round = 50
bst = xgb.train(params, dtrain, num_round)

# Membuat prediksi
y_pred = bst.predict(dtest)
accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi XGBoost Menggunakan Power Transformer adalah : {accuracy*100:.2f}%")

Akurasi XGBoost Menggunakan Power Transformer adalah : 83.61%


In [ ]:
# import KNeighbors ClaSSifier from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# instantiate the model
knn = KNeighborsClassifier(n_neighbors=3)


# fit the model to the training set
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(f"Akurasi KNN Menggunakan Power Transformer adalah :{np.sum(y_pred == y_test)/len(y_test) * 100}%")

Akurasi KNN Menggunakan Power Transformer adalah :80.32786885245902%
